In [ ]:
import matplotlib
import pandas as pd
import sys
import os
import scanpy as sc
import anndata as ad
import polars as pl

In [ ]:
specs5k_aggregated = pl.read_csv("/home/jovyan/share/data/analyses/martin/024_specs_all/HARMONY/harmony_specs_3k_2k/harm_results.csv", separator = ";").filter(pl.col("project").is_in(["specs3k", "specs2k"]))
specs5k_aggregated = specs5k_aggregated.rename({"" : "index"})

In [ ]:
features_specs5k_aggregated = pl.read_csv("/home/jovyan/share/data/analyses/martin/024_specs_all/HARMONY/DatasetForHarmony.csv", separator = ";")
features_specs5k_aggregated = features_specs5k_aggregated.rename({"" : "index"})

In [ ]:
specs5k_aggregated_all = specs5k_aggregated.join(features_specs5k_aggregated, on = ["index", "batch_id", "project"], how = "inner")

In [ ]:
meta_features = ["cbkid", "Metadata_Barcode", "batch_id", "project", "index"]
pca_features = [feat for feat in specs5k_aggregated_all.columns if "PC" in feat]
features = [feat for feat in specs5k_aggregated_all.columns if feat not in meta_features and feat not in pca_features]

In [ ]:
adata = ad.AnnData(X=specs5k_aggregated_all[features].to_pandas(), obs=specs5k_aggregated_all[meta_features].to_pandas())
pca = specs5k_aggregated_all[pca_features].to_numpy()
adata.obsm['X_pca'] = pca


In [ ]:
print("Starting scanpy!")
#sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=50)
sc.tl.paga(adata, groups="cbkid")
sc.pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata, init_pos='paga')
print("Embedding complete. Saving file!")

In [ ]:
sc.pl.umap(adata, color= "project")

In [ ]:
adata.write("aggregated_specs5k_embedding.h5ad")

In [ ]:
adata = ad.read_h5ad("aggregated_specs5k_embedding.h5ad")

In [ ]:
len(adata.obs["cbkid"].unique())

In [ ]:
adata = ad.read_h5ad("aggregated_specs5k_embedding_cmp.h5ad")

In [ ]:
adata.write_h5ad("specs5K_agg_cmpd.h5ad")

In [ ]:
adata.obs